# Informe de desempeño de PROESUR años 2015 - 2024

Este informe consta del desempeño del campus PROESUR desde el año 2015 hasta el año 2024. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
FOLDER = Path("data")  
OUTPUT = Path("Extracted_data")

In [3]:
def load_small_chunks(path: Path, col: str = "Cod_Estable",code: str = "05-02-0232-46") -> pd.DataFrame:
    '''
    Extracts only the data we want from each xlsx file 
    ''' 
    df = pd.read_excel(path, engine="openpyxl")
    df = df[df[col] == code]
    return df

def concat_dataframes(files: list[Path]) -> pd.DataFrame | None:
    df_list: list[pd.DataFrame] = []
    for f in files:
        df_list.append(load_small_chunks(f))
    try:
        result = pd.concat(df_list, ignore_index=True)
        return result
    except Exception as e:
        print(f"Error: {e}")
        return None
 

In [4]:
def load_small_chunks_multiple(path: Path, col: str = "Cod_Estable", code: str = "05-02-0232-46") -> pd.DataFrame:
    """Read first sheet, keep ALL columns, filter by the first alias that exists."""
    df = pd.read_excel(path, engine="openpyxl", dtype=str)  # keep as strings
    try: 
        df = df[df[col] == code]  
    except Exception:
        df = df[df[col] == code]
        return df
    else:
        return df

    

In [5]:

# for f in sorted(FOLDER.glob("*.xlsx"), reverse=True):
#     df = load_small_chunks_multiple(f)
#     if not df.empty:
#         df.to_csv(OUTPUT / f"{f.stem}_filtered.csv", index=False)
#     else:
#         print(f"{f.stem} was empty")


In [5]:
def search_by_value(path: Path, val:str, col:str) -> pd.DataFrame:
    df = pd.read_excel(path, engine="openpyxl")
    mask = df[col].astype(str).str.contains(val, case=False, na=False)
    return df.loc[mask]


def search_by_map(path: Path, query: dict[str, str], op: str = "AND") -> pd.DataFrame:
    """
    query: {"NOM_ESTAB": "INSTITUTO", "Municipio": "Guatemala"}
    op: "AND" or "OR"
    """
    df = pd.read_excel(path, engine="openpyxl", dtype=str)
    masks = []
    for col, val in query.items():
        if col in df.columns:
            m = df[col].astype(str).str.contains(val, case=False, na=False)
            masks.append(m)
    if not masks:
        return df.iloc[0:0]
    if op.upper() == "OR":
        mask = masks[0]
        for m in masks[1:]: mask = mask | m
    else:
        mask = masks[0]
        for m in masks[1:]: mask = mask & m
    return df.loc[mask]


In [7]:
# FILES = ["GRADUANDOS 2015.xlsx", "GRADUANDOS 2016.xlsx"]

# for fname in FILES:
#     file_path = FOLDER / fname
#     if not file_path.exists():
#         print(f"File not found: {file_path}")
#         continue
#     result = search_by_value(file_path, "INSTITUTO TECNOLOGICO PROESUR ", "NOM_ESTAB")
#     result.to_csv(OUTPUT / f"{fname}_filtered.csv", index=False)
#     print(f"{fname}: {len(result)} rows found")

In [8]:
# FILES = ["2019-Grad-Internet.xlsx", "2018_-_Grad_Internet.xlsx", "2017_Grad-Version_Internet.xlsx"]
# query = {"Cod_Muni": "0502", "Direc_Estable": "Km. 92.5 Finca Camantulul Carretera A Mazatenango"}
# for fname in FILES:
#     file_path = FOLDER / fname
#     if not file_path.exists():
#         print(f"File not found: {file_path}")
#         continue
#     result = search_by_map(file_path, query=query, op="AND")
#     result.to_csv(OUTPUT / f"{fname}_filtered.csv", index=False)
#     print(f"{fname}: {len(result)} rows found")

## Análisis de los datos obtenidos

In [ ]:

ALIASES = {

    # academic 
    "nivel_mate": [
        "DESEMPEÑO_MATEMÁTICAS", "Desempeño_Mate"
    ],
    "nivel_lectura": [
        "DESEMPEÑO_LECTURA", "Desempeño_Lect"
    ],  
    "logro_mate": [
        "LOGRO_MATEMÁTICAS", "Logro_Mate"
    ],
    "logro_lectura": [
        "LOGRO_LECTURA", "Logro_Lect"
    ],
    "periodos_semanales_mate" : [
        "MAT_PERIODOS_MATEMATICAS_SEMANA_Recodificada", "Mate_Periodos_Matematicas_Semana_RECO","Mate_Periodos_Matematicas_Semana_Reco"
    ],
    "periodos_semanales_lectura" : [
        "LEC_PERIODOS_LECTURA_SEMANA_Recodificada", "Lect_Periodos_Lectura_Semana_RECO", "Lect_Periodos_Lectura_Semana_Reco"
    ],
    # socioeconomic 
    "trabaja": [
       "ED_TRABAJA_ACTUALMENTE", "Ed_Trabaja_Actualmente"
    ],
    "acceso_internet": [
        "CC_SERVICIO_INTERNET", "CC_Servicio_Internet", "Sc_Servicio_Internet"
    ],
    "asistencia_primaria" : [
        "ED_ASISTIO_PREPRIMARIA" , "Ed_Asistio_Preprimaria"
    ],
    "sexo": [
        "GENERO", "Sexo_RECO", "Sexo_Reco"
    ],
    "padre_asistio_escuela" : [
        "FM_ASISTIO_ESCUELA_PAPA", "Fm_Asistio_Escuela_Papa"
    ],
    "grado_alcanzado_padre" : [
        "FM_GRADO_ALCANZO_PAPA_Recodificada", "Fm_Grado_Alcanzo_Papa_RECO"
    ],
    "madre_asistio_escuela" : [
        "FM_ASISTIO_ESCUELA_MAMA", "Fm_Asistio_Escuela_Mama"
    ],
    "grado_alcanzado_madre" : [
        "FM_GRADO_ALCANZO_MAMA_Recodificada", "Fm_Grado_Alcanzo_Mama_RECO"
    ],
    "grupo_etnico" : [
        "IE_IDENTIFICACION_ETNICA_Recodificada" , "Identificacion_Etnica_RECO"
    ],
    "rep_primaria" : [
        "ED_REPITIO_ALGUN_GRADO_PRIMARIA", "Ed_Repitio_Algun_Grado_Primaria", "Ed_Repitio_Algun_Grado"
    ]

}


In [7]:
def get_col(df: pd.DataFrame, key: str) -> str | None:
    """Return the actual column name for a logical key, or None if not present."""
    for name in ALIASES.get(key, []):
        if name in df.columns:
            return name
    return None

def dist_levels(df: pd.DataFrame, key: str) -> pd.Series:
    """
    Returns percentage distribution of categories in the column mapped by `key`.
    Example: key="nivel_mate" or "nivel_lectura".
    """
    col = get_col(df, key)
    if not col:
        print("No column found")
        return pd.Series(dtype=float)
    
    s = (
        df[col]
        .astype(str)
        .str.strip()
        .str.upper()
        .replace({"S/ DATO": None, "NA": None, "NAN": None})
    )
    return (s.value_counts(dropna=True, normalize=True) * 100).round(2)


# TODO modify to use dist_levels to map percentages
def pct_true(df: pd.DataFrame, key: str) -> float | None:
    """
    Percentage of 'true' / 'yes' / 1 for the column mapped by `key`.
    Returns None if column not present or no valid data.
    """
    col = get_col(df, key)
    if not col:
        print("Column was None")
        return None
    
    s = df[col].astype(str).str.strip().str.lower()
    true_vals = {"1"}
    false_vals = {"0"}
    
    mask_valid = s.isin(true_vals | false_vals)
    if not mask_valid.any():
        print("Not found any in mask")
        return None
    
    pct = (s.isin(true_vals) & mask_valid).mean() * 100
    return round(pct, 2)

def count_values(df: pd.DataFrame, column: str) -> pd.Series:
    """
    Counts each unique value in the specified column.
    Returns a Series sorted by count (descending).
    """
    col = get_col(df, column)
    if not col:
        return pd.Series(dtype=int)


    return df[col].value_counts(dropna=False)


def val_distribution(df: pd.DataFrame, key: str) -> pd.DataFrame:

    col = get_col(df, key)
    if not col:
        return pd.DataFrame()
    counts = df[col].value_counts(dropna=False)
    percents = (counts / counts.sum() * 100).round(2)
    return pd.DataFrame({"Count" : counts, "Percent" : percents}).sort_values("Count", ascending=False).reset_index(drop=True)

def value_dist(df: pd.DataFrame, col: str) -> pd.DataFrame:
    """
    Returns value/count/percent for col.
    Only non-empty distributions.
    """
    counts = df[col].value_counts(dropna=False)
    if counts.empty or counts.sum() == 0:
        return pd.DataFrame(columns=["value", "count", "percent"])

    perc = (counts / counts.sum() * 100).round(2)

    out = (
        pd.DataFrame({
            "value": counts.index.astype(str),
            "count": counts.values,
            "percent": perc.values,
        })
        .sort_values("count", ascending=False)
        .reset_index(drop=True)
    )
    return out

def analyze_file_to_metrics(path: Path) -> pd.DataFrame:
    """
    For a single CSV, compute distributions for all ALIASES keys.

    Returns a DataFrame with:
        variable, category, count, percent
    """
    df = pd.read_csv(path, dtype=str)

    rows = []

    for key in ALIASES.keys():
        col = get_col(df, key)
        if not col:
            continue

        dist = value_dist(df, col)
        if dist.empty:
            continue

        for _, r in dist.iterrows():
            val = str(r["value"]).strip()
            # skip NaN labels if they appear as 'nan'
            if val.lower() == "nan":
                continue

            rows.append({
                "variable": key,
                "category": val,
                "count": int(r["count"]),
                "percent": float(r["percent"]),
            })

    return pd.DataFrame(rows)



In [8]:
def run_all(
    folder: Path,
    filenames: list[str] | None = None,   # pass to test specific files
    pattern: str = "*.csv"                # used if filenames is None
):
    out_dir = folder / "metrics"
    out_dir.mkdir(exist_ok=True)

    # decide which CSVs to process
    if filenames is not None:
        csv_paths = []
        for name in filenames:
            p = folder / name
            if p.exists():
                csv_paths.append(p)
            else:
                print(f"⚠️ Missing file (skipped): {p}")
    else:
        csv_paths = sorted(folder.glob(pattern))

    for csv_path in csv_paths:
        metrics_df = analyze_file_to_metrics(csv_path)

        if metrics_df.empty:
            print(f"{csv_path.name}: no metrics found (check ALIASES or data).")
            continue

        out_path = out_dir / f"{csv_path.stem}__metrics.csv"
        metrics_df.to_csv(out_path, index=False)
        print(f"{csv_path.name}: wrote {out_path.name}")


In [10]:
CLEAN_PATH = Path("Extracted_data")

In [ ]:

early_reports: list[str] = ["GRADUANDOS 2015.xlsx_filtered.csv", "GRADUANDOS 2016.xlsx_filtered.csv"]

run_all(CLEAN_PATH, early_reports)

GRADUANDOS 2015.xlsx_filtered.csv: wrote GRADUANDOS 2015.xlsx_filtered__metrics.csv
GRADUANDOS 2016.xlsx_filtered.csv: wrote GRADUANDOS 2016.xlsx_filtered__metrics.csv


In [ ]:

early_reports: list[str] = ["2017_Grad-Version_Internet.xlsx_filtered.csv", "2018_-_Grad_Internet.xlsx_filtered.csv", "2019-Grad-Internet.xlsx_filtered.csv"]

run_all(CLEAN_PATH, early_reports)

2017_Grad-Version_Internet.xlsx_filtered.csv: wrote 2017_Grad-Version_Internet.xlsx_filtered__metrics.csv
2018_-_Grad_Internet.xlsx_filtered.csv: wrote 2018_-_Grad_Internet.xlsx_filtered__metrics.csv
2019-Grad-Internet.xlsx_filtered.csv: wrote 2019-Grad-Internet.xlsx_filtered__metrics.csv


In [13]:

early_reports: list[str] = ["2022-Grad-Internet_filtered.csv", "2023-Grad-Internet_filtered.csv", "2024-Grad-Internet_filtered.csv"]

run_all(CLEAN_PATH, early_reports)

2022-Grad-Internet_filtered.csv: wrote 2022-Grad-Internet_filtered__metrics.csv
2023-Grad-Internet_filtered.csv: wrote 2023-Grad-Internet_filtered__metrics.csv
2024-Grad-Internet_filtered.csv: wrote 2024-Grad-Internet_filtered__metrics.csv
